In [1]:
import mechanicalsoup
from dotenv import load_dotenv
import os

load_dotenv('.env')


browser = mechanicalsoup.StatefulBrowser()

def login(username, password):
    browser.open("https://members.foodcoop.com/services/login/")

    browser.select_form('form[action="/services/login/"]')
    browser["username"] = username
    browser["password"] = password
    resp = browser.submit_selected()

    if resp.url == "https://members.foodcoop.com/services/login/":
        raise Exception("Login failed")
    else:
        print("Login successful")

In [2]:
def logout():
    browser.open("https://members.foodcoop.com/services/logout/")

In [4]:
login(os.getenv("COOP_USERNAME"), os.getenv("COOP_PASSWORD"))

Login successful


In [ ]:
shifts = []
browser.open("https://members.foodcoop.com/services/shifts/")
#browser.open("https://members.foodcoop.com/services/shifts/1/0/0/2024-09-07/")

In [ ]:
while True:
    page = browser.get_current_page()
    
    shift_elements = page.find_all(class_='shift')
    
    for shift in shift_elements:
        time = shift.b.text
        shift.b.extract()
        type = shift.text.replace('🥕', '').strip()
        date = shift.parent.p.b.text.replace('\xa0', ' ')
        
        shifts.append({
            "date": date,
            "time": time,
            "type": type
        })
    
    print(f"Processed {len(shift_elements)} shifts. Total shifts: {len(shifts)}")
    
    try:
        browser.follow_link(text="Next Week →")
    except mechanicalsoup.LinkNotFoundError:
        print("No more weeks to process.")
        break

print(f"Total shifts collected: {len(shifts)}")